In [1]:
import chit

%load_ext autoreload
%autoreload 2

In [2]:
chat = chit.Chat.clone("foo2.json")

In [3]:
chat.log("tree")

[S_]dfac3dae──[U*]203da29b──[A_]dab57316── (master)
            │             ├─[A_]2ad8645b── (retry_response)
            │             └─ (new_branch*)
            └─[U_]ecf21cad──[A_]af5de4e2──[U_]1aeaedd1──[A_]afacbdeb──[U_]f05b7bd4──[A_]8ee7390f── (edit_prompt)
